### 학습 내용
>SVD 함수 사용법

>최소자승법(Least-Squares Solution) 함수 사용법

* SVD : U,s,VT = linalg.svd(A,compute_uv=True)
    * U,V : orthogonal, unitary(conjugate transpose($A^*$)와 동일한 행렬) / $\Sigma$ : singular values로 이루어진 행렬 
    * $A = U\Sigma V^T$
        * A = U@linalg.diagsvd(s,A.shape$[0]$,A.shape$[1]$)@VT
    * $A = U\Sigma V^*$
    * s : singular values 큰것부터 (0도 포함한) 1D array 
    * VT : V.T
    * compute_uv : s만 뽑을 지, uv도 뽑을지 결정

* Reduced SVD
    * SVD를 block matrix를 사용해서 계산(근사)
    * $A = U_rDV_r^T$ 
        * D : diagonal값이 nonzero singular values로 이루어진 대각 행렬
        * r : rank A # r = s.shape$[0]$ - sum(lx<1.e-10 for lx in s)
    * U$[:,:r]$@np.diag(s$[:r]$)@VT$[:r,:]$
        *  = U$[:,:r]$*s$[:r]$@VT$[:r,:]$
    
* Truncated SVD
    * reduced SVD에서의 D를 더욱 근사 (작은값들을 제외하고 줄임)
    * $A = U_rDV_r^T$ , $A^* = U_tD_tV_t^T$
    * U$[:,:t]$*s$[:t]$@VT$[:t,:]$
    * t = sum(lx > 1.e-3*s$[0]$ for lx in s) # e-3 : singular value가 1000이하면 무시
    
* Column Space와 Null Space를 구하는 법
    * ColA = linalg.orth(A, rcond = None) : A의 Column space의 basis를 매트릭스로 변환 
    * NullA = linalg.null_space(A, rcond = None) : Null space의 basis를 매트릭스로 변환 
        * rcond : 기본값 (var=2.22e-16)*가장 큰 singular value 이하는 무시 
    
* Pseudoinverse (Moore-Penrose inverse) : pinv_A = linalg.pinv(A)
    * $A^+=V_rD^{-1}U_r^T$
    * $A^+b$ : least-squares solution 
    * pinv_A = $A^+$
    * $V_r$ = VT$[:r,:]$.T
    * $D^{-1}$ = linalg.inv(D) 
        * D : np.diag(s$[:r]$)
    * $U_r^T$ = $U_r$.T 
        * $U_r$ : $U[:,:r]$

* 상대적으로 작은 Singular Value들을 무시하고 싶을때 
    * truncated SVD와 유사하게 근사
    * pinv_A = linalg.pinv(A,rcond=val) : 수두인버스 근사
        * rcond : 기본값 (var=2.22e-16)*가장 큰 singular value 이하는 무시

* Least-Squares Solution : x_hat, res, rank, s = linalg.lstsq(A,b,cond = None)
    * A는 m x n, Ax = b 일때 $\lVert b-A\hat x \rVert$를 최소로 하는 solution
    * pseudoinverse $A^+V_rD^{-1}U_r^T\to A^+b$ : least-squares solution
        * x_hat : solution
        * res : 오차 $\lVert b-A\hat x \rVert$
        * 

### code

In [2]:
import numpy as np
from scipy import linalg

In [8]:
# svd
A = np.array([[1,-1],[-2,2],[2,-2]])
U,s,VT = linalg.svd(A,compute_uv =True)
print(U,s,VT)
Sigma = linalg.diagsvd(s,A.shape[0],A.shape[1]) # s를 시그마 행렬로 표현
print(Sigma)
r = s.shape[0] - sum(np.allclose(lx,0) for lx in s)
reduced = U[:,:r]*s[:r]@VT[:r,:]
reduced

[[-0.33333333  0.66666667 -0.66666667]
 [ 0.66666667  0.66666667  0.33333333]
 [-0.66666667  0.33333333  0.66666667]] [4.24264069 0.        ] [[-0.70710678  0.70710678]
 [ 0.70710678  0.70710678]]
[[4.24264069 0.        ]
 [0.         0.        ]
 [0.         0.        ]]


array([[ 1., -1.],
       [-2.,  2.],
       [ 2., -2.]])

In [13]:
ColA = linalg.orth(A, rcond = None)
NullA = linalg.null_space(A, rcond = None)
print(ColA) # U의 정보
print(NullA) # VT의 정보를 담고있음

[[-0.33333333]
 [ 0.66666667]
 [-0.66666667]]
[[0.70710678]
 [0.70710678]]
